In [ ]:
import os
import warnings

import numpy as np
import pandas as pd
from matplotlib.gridspec import GridSpec
from sklearn.calibration import CalibrationDisplay
from sklearn.model_selection import train_test_split

warnings.filterwarnings("ignore")

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [ ]:
data_encoded = pd.read_csv("seer_encoded.csv")

In [ ]:
dataset = data_encoded.reset_index(drop=True)
N = dataset.shape[0]

time_column = "time"
event_column = "cod"
features = np.setdiff1d(dataset.columns, [time_column, event_column] ).tolist()

# Building training and testing sets
index_train, index_test = train_test_split( range(N), test_size = 0.1, random_state = 142857, stratify = dataset[event_column])
data_train = dataset.loc[index_train].reset_index( drop = True )
data_test  = dataset.loc[index_test].reset_index( drop = True )
N2 = data_train.shape[0]

index_train, index_val = train_test_split( range(N2), test_size = 0.1, random_state = 142857, stratify = data_train[event_column])
data_train = dataset.loc[index_train].reset_index( drop = True )
data_val  = dataset.loc[index_val].reset_index( drop = True )

# Creating the X, T and E inputs
X_train, X_test, X_val = data_train[features], data_test[features], data_val[features]
time_train, time_test, time_val = data_train[time_column], data_test[time_column], data_val[time_column]
event_train, event_test, event_val = data_train[event_column], data_test[event_column], data_val[event_column]

step = 5
print(X_train.shape)
print(X_test.shape)

In [ ]:
# RSF
# Expected CPU runtime: <10s
from sksurv.ensemble import RandomSurvivalForest
bool_indicator_train =  [bool(x) for x in event_train]
y_train_tuple =  c = np.array(list(zip(bool_indicator_train, time_train)), dtype=[("a", bool), ("b", float)])
rsf = RandomSurvivalForest(n_estimators=100, min_samples_split=6, 
                            min_samples_leaf=3, n_jobs=16)
rsf.fit(X_train, y_train_tuple)
rsf_func = rsf.predict_survival_function(X_test)
# survprob_rsf= np.array(1-rsf.predict_survival_function(X_test, times = np.arange(time_test.min(),time_test.max(),step)).T)

In [ ]:
# CoxPH
# Expected CPU runtime: <10s
from sksurv.linear_model import CoxPHSurvivalAnalysis
bool_indicator_train =  [bool(x) for x in event_train]
y_train_tuple =  c = np.array(list(zip(bool_indicator_train, time_train)), dtype=[("a", bool), ("b", float)])
CoxPH = CoxPHSurvivalAnalysis(alpha=0.1,verbose=0) # alpha=0 causes singular matrix error

CoxPH.fit(X_train, y_train_tuple)
coxph_func = CoxPH.predict_survival_function(X_test)

In [ ]:
# DeepSurv
# Expected CPU runtime: <30s
from pycox.models import CoxPH
import torchtuples as tt
import torch
# set default tensor type
# torch.set_default_tensor_type(torch.cuda.FloatTensor)
batch_size = 4096
epochs = 10000
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


y_train_tuple = (time_train.to_numpy().astype(np.float32), event_train.to_numpy().astype(np.float32))
y_val_tuple = (time_val.to_numpy().astype(np.float32), event_val.to_numpy().astype(np.float32))
val = X_val.to_numpy().astype(np.float32), y_val_tuple
in_features = X_train.shape[1]
num_nodes = [32, 32, 32]
out_features = 1
batch_norm = True
dropout = 0.0
output_bias = False

net = tt.practical.MLPVanilla(in_features, num_nodes, out_features, batch_norm, dropout, output_bias=output_bias)
deepsurv = CoxPH(net, tt.optim.Adam, device=device)
deepsurv.optimizer.set_lr(0.001)
callbacks = [tt.callbacks.EarlyStopping(patience=100)]
verbose = False
log = deepsurv.fit(X_train.to_numpy().astype(np.float32), y_train_tuple, batch_size, epochs, callbacks, verbose,
                val_data=val, val_batch_size=batch_size)
_ = deepsurv.compute_baseline_hazards()
deepsurv_pred = deepsurv.predict_surv_df(X_test.to_numpy().astype(np.float32))

In [ ]:
# CoxCC
# Expected CPU runtime: <30s
from pycox.models import CoxCC
import torchtuples as tt
import torch
batch_size = 4096
epochs = 10000

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

y_train_tuple = (time_train.to_numpy().astype(np.float32), event_train.to_numpy().astype(np.float32))
y_val_tuple = (time_val.to_numpy().astype(np.float32), event_val.to_numpy().astype(np.float32))
val = X_val.to_numpy().astype(np.float32), y_val_tuple
in_features = X_train.shape[1]
num_nodes = [32, 32, 32]
out_features = 1
batch_norm = True
dropout = 0.0
output_bias = False

net = tt.practical.MLPVanilla(in_features, num_nodes, out_features, batch_norm, dropout, output_bias=output_bias)
coxcc = CoxCC(net, tt.optim.Adam, device=device)
coxcc.optimizer.set_lr(0.001)
callbacks = [tt.callbacks.EarlyStopping(patience=100)]
verbose = False
log = coxcc.fit(X_train.to_numpy().astype(np.float32), y_train_tuple, batch_size, epochs, callbacks, verbose,
                val_data=val, val_batch_size=batch_size)
_ = coxcc.compute_baseline_hazards()
coxcc_pred = coxcc.predict_surv_df(X_test.to_numpy().astype(np.float32))

In [ ]:
# DeepHit
# Expected CPU runtime: 11m19s
from pycox.models import DeepHitSingle
import torchtuples as tt
import torch

num_durations = 250
batch_size = 4096*4
epochs = 10000
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

labtrans = DeepHitSingle.label_transform(num_durations)
y_train_tuple = labtrans.fit_transform(time_train.to_numpy().astype(np.float32), event_train.to_numpy().astype(np.float32))
y_val_tuple = labtrans.fit_transform(time_val.to_numpy().astype(np.float32), event_val.to_numpy().astype(np.float32))
val = X_val.to_numpy().astype(np.float32), y_val_tuple

in_features = X_train.shape[1]
num_nodes = [32, 32, 32]
out_features = num_durations
batch_norm = True
dropout = 0.0
net = tt.practical.MLPVanilla(in_features, num_nodes, out_features, batch_norm, dropout)
deephit = DeepHitSingle(net, tt.optim.Adam, alpha=0.2, sigma=0.1, duration_index=labtrans.cuts, device=device)
deephit.optimizer.set_lr(0.001)
callbacks = [tt.callbacks.EarlyStopping(patience=100)]
verbose = False
log = deephit.fit(X_train.to_numpy().astype(np.float32), y_train_tuple, batch_size, epochs, callbacks, verbose, val_data=val, val_batch_size=batch_size)
deephit_pred = deephit.predict_surv_df(X_test.to_numpy().astype(np.float32))

In [ ]:
# PCHazard
# Expected CPU runtime: 6m46s
from pycox.models import PCHazard
import torchtuples as tt
import torch

batch_size = 4096*4
epochs = 10000
num_durations = 250
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

labtrans = PCHazard.label_transform(num_durations)
y_train_tuple = labtrans.fit_transform(time_train.to_numpy().astype(np.float32), event_train.to_numpy().astype(np.float32))
y_val_tuple = labtrans.fit_transform(time_val.to_numpy().astype(np.float32), event_val.to_numpy().astype(np.float32))
val = X_val.to_numpy().astype(np.float32), y_val_tuple

in_features = X_train.shape[1]
num_nodes = [32, 32, 32]
out_features = labtrans.out_features
batch_norm = False
dropout = 0.0
net = tt.practical.MLPVanilla(in_features, num_nodes, out_features, batch_norm, dropout)
pchazard = PCHazard(net, tt.optim.Adam, device=device)
pchazard.optimizer.set_lr(0.0001)
callbacks = [tt.callbacks.EarlyStopping(patience=200)]
verbose = False
log = pchazard.fit(X_train.to_numpy().astype(np.float32), y_train_tuple, batch_size, epochs, callbacks, verbose, val_data=val, val_batch_size=batch_size)
pchazard_pred = pchazard.predict_surv_df(X_test.to_numpy().astype(np.float32))

In [ ]:
# DCSurvival
# Expected CPU runtime: ??
import sys
import torch
from dcsurvival.dirac_phi import DiracPhi
from dcsurvival.survival import SurvivalCopula_sumofull
import torch.optim as optim
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader, TensorDataset

torch.set_num_threads(24)
torch.set_default_tensor_type(torch.DoubleTensor)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

load_previous = True
depth = 2
widths = [100, 100]
lc_w_range = (0, 1.0)
shift_w_range = (0., 2.0)
num_epochs = 100000
batch_size = 4096*4
early_stop_epochs = 500

times_tensor_train = torch.tensor(time_train.to_numpy(), dtype=torch.float64).to(device)
event_indicator_tensor_train = torch.tensor(event_train.to_numpy(), dtype=torch.float64).to(device)
covariate_tensor_train = torch.tensor(X_train.to_numpy(), dtype=torch.float64).to(device)

times_tensor_val = torch.tensor(time_val.to_numpy(), dtype=torch.float64).to(device)
event_indicator_tensor_val = torch.tensor(event_val.to_numpy(), dtype=torch.float64).to(device)
covariate_tensor_val = torch.tensor(X_val.to_numpy(), dtype=torch.float64).to(device)

dataset = TensorDataset(covariate_tensor_train, times_tensor_train, event_indicator_tensor_train)
val_dataset = TensorDataset(covariate_tensor_val, times_tensor_val, event_indicator_tensor_val)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

phi = DiracPhi(depth, widths, lc_w_range, shift_w_range, device, tol = 1e-14).to(device)
dcs_model = SurvivalCopula_sumofull(phi, device = device, num_features=X_train.shape[1], 
                                tol=1e-14, hidden_size = 32)
dcs_model.to(device)

if load_previous == True:
    checkpoint = torch.load("/home/SurvivalACNet_sumo/checkpoints/weight.pth")
    dcs_model.load_state_dict(checkpoint["model_state_dict"])
optimizer = optim.AdamW([{"params": dcs_model.sumo_e.parameters(), "lr": 1e-6},
                        {"params": dcs_model.sumo_c.parameters(), "lr": 1e-6},
                        {"params": dcs_model.phi.parameters(), "lr": 1e-6}
                    ])
# Train the model
best_val_loglikelihood = float("-inf")
epochs_no_improve = 0 
for epoch in tqdm(range(num_epochs)):
# for epoch in range(num_epochs):
    optimizer.zero_grad()
    logloss = dcs_model(covariate_tensor_train, times_tensor_train, event_indicator_tensor_train, max_iter = 10000)
    (-logloss).backward()
    optimizer.step()
    # for covariates, times, events in dataloader:  # iterate over batches 
    #     optimizer.zero_grad()
    #     logloss = dcs_model(covariates, times, events, max_iter = 10000)
    #     (-logloss).backward() 
    #     optimizer.step()
    if epoch % 100 == 0:
        val_loglikelihood = dcs_model(covariate_tensor_val, times_tensor_val, event_indicator_tensor_val, max_iter = 10000)
        print("epoch", epoch, "train likelihood", logloss.item(),"val likelihood", val_loglikelihood.item())
        if val_loglikelihood > (best_val_loglikelihood ):
            best_val_loglikelihood = val_loglikelihood
            epochs_no_improve = 0
            torch.save({"epoch": epoch, "model_state_dict": dcs_model.state_dict(),"loss": best_val_loglikelihood,
                        }, "/home/SurvivalACNet_sumo/checkpoints/weight.pth")
        else:
            epochs_no_improve = epochs_no_improve + 100
    # Early stopping condition
    if epochs_no_improve == early_stop_epochs:
        # print("Early stopping triggered at epoch: %s" % epoch)
        break
# load the best model
checkpoint = torch.load("/home/SurvivalACNet_sumo/checkpoints/weight.pth")
dcs_model.load_state_dict(checkpoint["model_state_dict"])

In [ ]:
setp = 5

print(X_test.shape)
times = np.arange(time_test.min(), time_test.max(), step)

# times_tensor = torch.tensor(times).to(device)
# covariate_tensor_test = torch.tensor(X_test.to_numpy(), dtype=torch.float64).to(device)
# survprob_matrix_list = []
# for time in times_tensor:
#     temp = 1- dcs_model.survival(time, covariate_tensor_test).cpu().detach().numpy()
#     survprob_matrix_list.append(temp)
# survprob_matrix = np.vstack(survprob_matrix_list)
# survprob_dcsurvival = list(survprob_matrix.transpose())

survprob_rsf = [1-rsf_func[i](times) for i in range(X_test.shape[0])]
survprob_coxph = [1-coxph_func[i](times) for i in range(X_test.shape[0])]

survprob_deepsurv = [[1-deepsurv_pred[i][time] for time in deepsurv_pred.index] for i in range(X_test.shape[0])]
survprob_pchazard = [[1-pchazard_pred[i][time] for time in pchazard_pred.index] for i in range(X_test.shape[0])]
survprob_deephit = [[1-deephit_pred[i][time] for time in deephit_pred.index] for i in range(X_test.shape[0])]
survprob_coxcc = [[1-coxcc_pred[i][time] for time in coxcc_pred.index] for i in range(X_test.shape[0])]

### Plotting

In [ ]:
# y_l is the event time
y_l = []
prob_rsf = []
prob_coxph=[]
prob_deepsurv = []
prob_deephit = []
prob_pchazard = []
prob_coxcc = []
# prob_dcsurvival = []

for j in range(len(survprob_coxph)): #
    k = 0
    for i in np.arange(time_test.min(),time_test.max(),step): # from min to max of test time, the number of steps
        if k < len(survprob_coxph[0]):
            # if i <= time_test[j]:
            #     print(j)
            #     y_l.append(0)
            # else:
            #     y_l.append(1)
            prob_rsf.append(survprob_rsf[j][k])
            prob_coxph.append(survprob_coxph[j][k])
            k+=1
        else:
            break

for j in range(len(survprob_deepsurv)): #
    k = 0
    for i in np.arange(time_test.min(),time_test.max(),step): # from min to max of test time, the number of steps
        if k < len(survprob_deepsurv[0]):
            if i <= time_test[j]:
                y_l.append(0)
            else:
                y_l.append(1)
            prob_deepsurv.append(survprob_deepsurv[j][k])

            prob_coxcc.append(survprob_coxcc[j][k])
            prob_deephit.append(survprob_deephit[j][k])
            prob_pchazard.append(survprob_pchazard[j][k])
            # prob_dcsurvival.append(survprob_dcsurvival[j][k])
            k+=1
        else:
            break

y_l = np.array(y_l)
prob_rsf = np.array(prob_rsf)
prob_coxph = np.array(prob_coxph)
prob_deepsurv = np.array(prob_deepsurv)
prob_deephit = np.array(prob_deephit)
prob_pchazard = np.array(prob_pchazard)
prob_coxcc = np.array(prob_coxcc)
# prob_dcsurvival = np.array(prob_dcsurvival)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1,1, figsize=(8, 8))
disp = CalibrationDisplay.from_predictions(y_l,prob_coxph, ax = ax, name="CoxPH", marker="s")
disp = CalibrationDisplay.from_predictions(y_l,prob_rsf, ax = ax, name="RSF", marker="o")
disp = CalibrationDisplay.from_predictions(y_l,prob_deepsurv, ax = ax, name="DeepSurv", marker="^")
disp = CalibrationDisplay.from_predictions(y_l,prob_coxcc, ax = ax, name="CoxCC", marker="H")
disp = CalibrationDisplay.from_predictions(y_l,prob_deephit, ax = ax, name="DeepHit", marker="p")
disp = CalibrationDisplay.from_predictions(y_l,prob_pchazard, ax = ax, name="PCHazard", marker="*")
# disp = CalibrationDisplay.from_predictions(y_l,prob_dcsurvival, ax = ax, name="DCSurvival", marker="D")

ax.set_xlabel("Mean Predicted Probability", fontsize=14)
ax.set_ylabel("Fraction of Positives", fontsize=14)
ax.legend(fontsize=10)

plt.plot()